In [37]:
import pandas as pd

df = pd.read_csv("data/FNBr-sent-ces.csv")
df = df.sort_values(by=["cxnName", "startChar", "endChar"])

In [72]:
from collections import defaultdict

dataset = defaultdict(list)

for cxn_name, group in df.groupby(by=["cxnName"]):
    for _, ces in group.groupby(by=["text"]):
        sent = ces.iloc[0]["text"]
        ce_string = ", ".join(ces["ceName"].tolist())
        prompt = f"""Construction Grammar is a theory in Linguistics which states that form-meaning pairs \
called constructions are the basic units of language. In Brazilian Portuguese, the {cxn_name[0]} construction \
has {len(ces)} construction elements: {ce_string}.
Consider the following sentence: \"{sent}\". Answer which spans of the sentence represent different construction \
elements. Reply in a JSON format, with the construction element as the key and the span as the value."""

        dataset[cxn_name[0]].append({
            "prompt": prompt,
            "answer": ces.apply(lambda r: r["text"][r["startChar"]:r["endChar"]+1], axis=1).tolist()
        })

In [75]:
import json

with open("construct-prompts.json", "w") as fp:
    json.dump(dataset, fp)